# A SAT-Based Approach for Mining Association Rules

The following program takes in input the *Congressional Voting Records* dataset (https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records) and computes its Association Rules using SAT, as proposed by Abdelhamid Boudane et al.

In [1]:
import numpy as np
import pandas as pd
from pysat.solvers import Minisat22
from itertools import combinations
from pysat.pb import *
import time
import datetime

In [2]:
alpha = 0.2 #support threshold - set it s.t. size*alpha > 0
beta = 1 #confidence threshold
size = 50
ARM = Minisat22()

## Data preparation

In [3]:
colnames = ["class_name","handicapped_infants","water_project_cost_sharing","adoption_of_the_budget_resolution",
            "physician_fee_freeze","el_salvador_aid","religious_groups_in_schools","anti_satellite_test_ban",
            "aid_to_nicaraguan_contras","mx_missile","immigration","synfuels_corporation_cutback",
            "education_spending","superfund_right_to_sue","crime","duty_free_exports",
            "export_administration_act_south_africa"]

data = pd.read_csv("/home/laura/Scrivania/KDMproj/house-votes-84.txt", sep = ',',na_values = '?', header = None, 
                   names = colnames)
data = pd.DataFrame(data)


data["republican"] = 0
data.loc[data.class_name=="republican","republican"] = pd.Series(np.ones(len(data.class_name=="republican"),dtype=int))
del data["class_name"]
del colnames[0]

items = list(data.columns)

data.replace(to_replace = "n", value = 0, inplace = True)
data.replace(to_replace = "y", value = 1, inplace = True)
data.fillna("-1",inplace = True)
data = data.astype("int32")

print(len(data))
data.head()

435


,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa,republican
0,0,1,0,1,1,1,0,0,0,1,-1,1,1,1,0,1,1
1,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,-1,1
2,-1,1,1,-1,1,1,0,0,0,0,1,0,1,1,0,0,0
3,0,1,1,0,-1,1,0,0,0,0,1,0,1,0,0,1,0
4,1,1,1,0,1,1,0,0,0,0,1,-1,1,1,1,1,0


In [4]:
data_small = data.iloc[:size,]

In [5]:
#creates a dictionary that associates to each item "a" a unique integer value.

items_ID = dict(zip([x for x in range(1,len(items)+1)],items))
items_ID

{1: 'handicapped_infants',
 2: 'water_project_cost_sharing',
 3: 'adoption_of_the_budget_resolution',
 4: 'physician_fee_freeze',
 5: 'el_salvador_aid',
 6: 'religious_groups_in_schools',
 7: 'anti_satellite_test_ban',
 8: 'aid_to_nicaraguan_contras',
 9: 'mx_missile',
 10: 'immigration',
 11: 'synfuels_corporation_cutback',
 12: 'education_spending',
 13: 'superfund_right_to_sue',
 14: 'crime',
 15: 'duty_free_exports',
 16: 'export_administration_act_south_africa',
 17: 'republican'}

## Encoding and insertion 

$x_a$: if the item $a$ is in the set $X$ or not. $a$ can be of the kind $Yes$ or $No$ (e.g., republican can be true or false. two variables $x_{rep}=Yes$ and $x_{rep}=No$ will be created). when the i-th item is true-valued, the encoding will be computed with the a's ID, when false with its double.

$y_a$: if the item $a$ is in the set $Y$ or not. same rules as before, but considering a's ID + 17

$p_i$: if the i-th transaction contains the set $X$.

$q_i$: if the i-th transaction contains the $X \cup Y$.

In [6]:
def x(a):
    return a

In [7]:
def y(a):
    return len(data_small.columns)*2+a

In [8]:
def p(i):
    return len(data_small.columns)*4+i

In [9]:
def q(i):
    return len(data_small)+len(data_small.columns)*4+i

In [10]:
# Start point
start_time = time.time()
print("Start time: ", datetime.datetime.now())

Start time:  2019-10-03 12:26:55.886177


### Rule 1

In [11]:
for a in range(1,2*len(data_small.columns)+1):
    ARM.add_clause([-x(a),-y(a)])

### Rule 2

In [12]:
def rule_21(ind, k):
    for i in range(1,np.shape(data_small)[0]+1):
        l = [x(a) for j,a in enumerate(ind) if data.iloc[i-1,j] != k] 
        l.append(p(i))
        ARM.add_clause(l)
        #print(l)
    
rule_21(range(1,len(data_small.columns)+1),1)
rule_21(range(len(data_small.columns)+1,2*len(data_small.columns)+1),0)

In [13]:
def rule_22(ind, k):
    for i in range(1,np.shape(data_small)[0]+1):
        for j,a in enumerate(ind):
            if data_small.iloc[i-1,j] != k :  
                ARM.add_clause([-x(a),-p(i)])
                #print([-x(a),-p(i)])

rule_22(range(1,len(data_small.columns)+1),1)
rule_22(range(len(data_small.columns)+1,2*len(data_small.columns)+1),0)

### Rule 3

In [14]:
def rule_31(ind,k):
    for i in range(1,np.shape(data_small)[0]+1):
        l = [y(a) for j,a in enumerate(ind) if data_small.iloc[i-1,j] != k] 
        l.append(-p(i))
        l.append(q(i))
        ARM.add_clause(l)
        ARM.add_clause([-q(i),p(i)])
        #print(l+[-q(i),p(i)])
        
rule_31(range(1,len(data_small.columns)+1),1)
rule_31(range(len(data_small.columns)+1,2*len(data_small.columns)+1),0)

In [15]:
def rule_32(ind, k):
    for i in range(1,np.shape(data_small)[0]+1):
        for j,a in enumerate(ind):
            if data_small.iloc[i-1,j] != k :  
                ARM.add_clause([-y(a),-q(i)])
                #print([-y(a),-q(i)])

rule_32(range(1,len(data_small.columns)+1),1)
rule_32(range(len(data_small.columns)+1,2*len(data_small.columns)+1),0)

### Rule 4

In [16]:
cnf = PBEnc.atleast(lits = [q(i) for i in range(1,np.shape(data_small)[0]+1)], 
                    bound = int(len(data_small)*alpha))

for clause in cnf.clauses:
    ARM.add_clause(clause)
    #print(clause)
    
print("\n{} variables currently added \n".format(ARM.nof_vars()))


626 variables currently added 



### Rule 5

In [17]:
cnf = PBEnc.atleast(lits = [q(i) for i in range(1,np.shape(data_small)[0]+1)]+[p(i) for i in range(1,np.shape(data_small)[0]+1)],
                    weights = list(np.repeat(100,np.shape(data_small)[0]))+list(np.repeat(int(-beta*100),np.shape(data_small)[0])),
                    bound = 0, top_id = ARM.nof_vars())
for clause in cnf.clauses:
    ARM.add_clause(clause)


3176 variables currently added 



### Rule 6

In [18]:
#not in the paper - to avoid empty sets as X or Y

X = [ x(a) for a in range(1,2*len(data_small.columns)+1)]
Y = [ y(a) for a in range(1,2*len(data_small.columns)+1)]

ARM.add_clause(X)
ARM.add_clause(Y)

In [19]:
print("\n{} variables currently used \n".format(ARM.nof_vars()))

print("\n{} clauses currently added \n".format(ARM.nof_clauses()))


3176 variables currently used 


7845 clauses currently added 



## Association rule extraction

In [20]:
print(ARM.solve())
if not ARM.solve(): print(ARM.get_core())

res = []

while ARM.solve():
    model = np.array(ARM.get_model())
    model = model[abs(model) <= len(data_small.columns)*4] #extracting the rule
    ARM.add_clause([-int(model[i]) for i in range(len(model))])
    #print(model)
    res.append(model[model>0])
    
print(ARM.nof_clauses())
res.sort(key = len)
print("\n{} rules found \n".format(len(res)))
res


True
1728


[array([ 4, 20, 39]),
 array([ 3,  8, 22, 55]),
 array([ 3,  8, 22, 68]),
 array([ 3,  8, 22, 55, 68]),
 array([ 3,  8, 22, 34, 55]),
 array([ 3,  8, 21, 22, 68])]

In [21]:
#finds models with minimal X, when Y is the same

n = len(data_small.columns)

min_models_pred = []
minimality = [True for i in range(len(res))]

for i in range(len(res)-1):
    for j in range(i+1,len(res)):
        if set(res[i][res[i]>2*n]) == set(res[j][res[j]>2*n]) and set(res[i][res[i]<=2*n]) < set(res[j][res[j]<=2*n]):
            minimality[j] = False
            
for i, x in enumerate(minimality):
    if x: min_models_pred.append(res[i])

min_models_pred
    

[array([ 4, 20, 39]),
 array([ 3,  8, 22, 55]),
 array([ 3,  8, 22, 68]),
 array([ 3,  8, 22, 55, 68])]

In [22]:
#finds models with maximal Y, when X is the same

models = []
maximality = [True for i in range(len(min_models_pred))]

for i in range(len(min_models_pred)-1,0,-1):
    for j in range(i-1,-1,-1):
        if set(min_models_pred[i][min_models_pred[i]<=2*n]) == set(min_models_pred[j][min_models_pred[j]<=2*n]) and set(min_models_pred[i][min_models_pred[i]>2*n]) > set(min_models_pred[j][min_models_pred[j]>2*n]):
            maximality[j] = False
            
for i, x in enumerate(maximality):
    if x: models.append(res[i])

models

[array([ 4, 20, 39]), array([ 3,  8, 22, 55, 68])]

In [23]:
decod_rules = open("/home/laura/Scrivania/KDMproj/association_rules_yn.txt", "w")

for model in models:
    
    decod_rules.write("X = { Yes:")
    decod_rules.write(" "+str([items_ID[code] for code in model[model<=n]]))
    decod_rules.write(" No: "+str([items_ID[code] for code in model[(model>n) & (model<=2*n)]-n]))
    decod_rules.write("} \nY = { Yes:")
    decod_rules.write(" "+str([items_ID[code] for code in model[(model>2*n) & (model<=3*n)]-2*n]))
    decod_rules.write(" No: "+str([items_ID[code] for code in model[(model>3*n) & (model<=4*n)]-3*n]))
    decod_rules.write(" } \n\n")
        
decod_rules.close()

In [24]:
# End point
end_time = time.time()
uptime = end_time - start_time
human_uptime = datetime.timedelta(seconds=uptime)

print("End time: ", datetime.datetime.now())
print("Uptime :" ,human_uptime)

End time:  2019-10-03 12:26:57.477748
Uptime : 0:00:01.591533
